In [1]:
import findspark 
findspark.init("/Users/jean/spark-2.4.4-bin-hadoop2.7")

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [3]:
spark = SparkSession.builder.appName('trees').getOrCreate()

In [4]:
df = spark.read.csv("College.csv", inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol="features")

In [8]:
output = assembler.transform(df)

In [9]:
indexer = StringIndexer(inputCol="Private", outputCol="Private_index")


In [10]:
output_fixed = indexer.fit(output).transform(output)

In [11]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- Private_index: double (nullable = false)



In [12]:
final_data = output_fixed.select(["features", "Private_index"])

In [13]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [14]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [15]:
from pyspark.ml.pipeline import Pipeline

In [16]:
dtc = DecisionTreeClassifier(labelCol="Private_index", featuresCol="features")

In [40]:
gbt = GBTClassifier(labelCol="Private_index", featuresCol="features")

In [41]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [42]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [29]:
my_binary_eval = BinaryClassificationEvaluator(labelCol="Private_index")

In [30]:
print('DTC')
my_binary_eval.evaluate(dtc_preds)

DTC


0.9218923418423973

In [43]:
print('RFC')
my_binary_eval.evaluate(rfc_preds)

RFC


0.9757676655567883

In [34]:
##This is not the ideal for GBT, because it does not have a rawpredictioncol
gbt_preds.printSchema()
print('GBT')
my_binary_eval.evaluate(gbt_preds)

root
 |-- features: vector (nullable = true)
 |-- Private_index: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

GBT


0.9530151683314838

In [35]:
my_binary_eval_2 = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Private_index")

In [36]:
print('GBT')
my_binary_eval_2.evaluate(gbt_preds)

GBT


0.8876248612652607

In [46]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [47]:
acc_eval = MulticlassClassificationEvaluator(labelCol="Private_index", metricName="accurary")

In [ ]:
print('RFC Accuracy')
acc_eval.evaluate()